In [1]:
import uproot
import matplotlib.pyplot as plt
import polars as pl
import pandas as pd
from scipy.stats import binned_statistic, binned_statistic_2d
import numpy as np
plt.rcParams.update({'font.size': 16})

In [2]:
import awkward as ak
import uproot
import polars as pl
import numpy as np

def N_exp(df):
    ret = np.zeros((7, 3))
    for i, pdg in enumerate([12, 14, 16, -12, -14, -16, None]):
        for j, isCC in enumerate([1, 0, None]):
            selected = df
            if pdg != None:
                selected = df.filter(
                    pl.col('nuPDG') == pdg
                )
            if isCC != None:
                selected = selected.filter(
                    pl.col('isCC') == isCC
                )
            ret[i, j] = np.sum(selected['final_weight'].to_numpy())
    return ret

def PrintTable(df):
    N_s = N_exp(df.filter(pl.col('Ebin') == pl.lit("Sub-GeV")))
    N_m = N_exp(df.filter(pl.col('Ebin') == pl.lit("Multi-GeV")))
    N_h = N_exp(df.filter(pl.col('Ebin') == pl.lit("High-GeV")))
    N = N_exp(df)

    lims = [1., 10.]

    Solcycle='Solar Maximum'
    OscParameters='NuFIT 5.2'
    Model='AR2320i'

    caption = 'Expected number of neutrino interactions in DUNE FD using the Honda fluxes at %s, %s nuclear model and %s oscillation parameters.'%(Solcycle, Model, OscParameters)
    label="label"
    print("\\begin{table}[h!]\n\\centering\n\\label{tab:%s}\n\\begin{tabular}{ccccccccccccc}"%(label))
    print("\\multicolumn{1}{l}{} & \\multicolumn{12}{c}{\\textbf{Totals per 10 kt $\\cdot$ yr}} \\\\ \\cline{2-13}")
    print("\\multicolumn{1}{c|}{} & \\multicolumn{12}{c|}{%s, %s, %s} \\\\ \\cline{2-13}"%(Solcycle, OscParameters, Model)) 
    print("\\multicolumn{1}{c|}{} & \\multicolumn{3}{c|}{\\begin{tabular}[c]{@{}c@{}}Sub-GeV\\\\ $[0.1-%.1f]\\,$GeV\\end{tabular}} & \\multicolumn{3}{c|}{\\begin{tabular}[c]{@{}c@{}}Multi-GeV\\\\ $[%.1f-%.1f]\\,$GeV\\end{tabular}} & \\multicolumn{3}{c|}{\\begin{tabular}[c]{@{}c@{}}High-GeV\\\\ $[%.1f-100.0]\\,$GeV\\end{tabular}} & \\multicolumn{3}{c|}{\\begin{tabular}[c]{@{}c@{}}Total\\\\ $[0.1-100.0]\\,$GeV\\end{tabular}} \\\\ \\cline{2-13} "%(lims[0], lims[0], lims[1], lims[1]))
    print("\\multicolumn{1}{c|}{} & \\multicolumn{1}{c|}{CC} & \\multicolumn{1}{c|}{NC} & \\multicolumn{1}{c|}{Total} & \\multicolumn{1}{c|}{CC} & \\multicolumn{1}{c|}{NC} & \\multicolumn{1}{c|}{Total} & \\multicolumn{1}{c|}{CC} & \\multicolumn{1}{c|}{NC} & \\multicolumn{1}{c|}{Total} & \\multicolumn{1}{c|}{CC} & \\multicolumn{1}{c|}{NC} & \\multicolumn{1}{c|}{Total} \\\\ \\hline")
    print("\\multicolumn{1}{|c|}{$\\nu_e$} & \\multicolumn{1}{c|}{%.1f} & \\multicolumn{1}{c|}{%.1f} & \\multicolumn{1}{c|}{%.1f} & \\multicolumn{1}{c|}{%.1f} & \\multicolumn{1}{c|}{%.1f} & \\multicolumn{1}{c|}{%.1f} & \\multicolumn{1}{c|}{%.2f} & \\multicolumn{1}{c|}{%.1f} & \\multicolumn{1}{c|}{%.1f} & \\multicolumn{1}{c|}{%.1f} & \\multicolumn{1}{c|}{%.1f} & \\multicolumn{1}{c|}{%.1f} \\\\ \\hline"%(N_s[0, 0], N_s[0, 1], N_s[0, 2], N_m[0, 0], N_m[0, 1], N_m[0, 2], N_h[0, 0], N_h[0, 1], N_h[0, 2], N[0, 0], N[0, 1], N[0, 2]))
    print("\\multicolumn{1}{|c|}{$\\nu_\\mu$} & \\multicolumn{1}{c|}{%.1f} & \\multicolumn{1}{c|}{%.1f} & \\multicolumn{1}{c|}{%.1f} & \\multicolumn{1}{c|}{%.1f} & \\multicolumn{1}{c|}{%.1f} & \\multicolumn{1}{c|}{%.1f} & \\multicolumn{1}{c|}{%.2f} & \\multicolumn{1}{c|}{%.1f} & \\multicolumn{1}{c|}{%.1f} & \\multicolumn{1}{c|}{%.1f} & \\multicolumn{1}{c|}{%.1f} & \\multicolumn{1}{c|}{%.1f} \\\\ \\hline"%(N_s[1, 0], N_s[1, 1], N_s[1, 2], N_m[1, 0], N_m[1, 1], N_m[1, 2], N_h[1, 0], N_h[1, 1], N_h[1, 2], N[1, 0], N[1, 1], N[1, 2]))
    print("\\multicolumn{1}{|c|}{$\\nu_\\tau$} & \\multicolumn{1}{c|}{%.1f} & \\multicolumn{1}{c|}{%.1f} & \\multicolumn{1}{c|}{%.1f} & \\multicolumn{1}{c|}{%.1f} & \\multicolumn{1}{c|}{%.1f} & \\multicolumn{1}{c|}{%.1f} & \\multicolumn{1}{c|}{%.2f} & \\multicolumn{1}{c|}{%.1f} & \\multicolumn{1}{c|}{%.1f} & \\multicolumn{1}{c|}{%.1f} & \\multicolumn{1}{c|}{%.1f} & \\multicolumn{1}{c|}{%.1f} \\\\ \\hline"%(N_s[2, 0], N_s[2, 1], N_s[2, 2], N_m[2, 0], N_m[2, 1], N_m[2, 2], N_h[2, 0], N_h[2, 1], N_h[2, 2], N[2, 0], N[2, 1], N[2, 2]))
    print("\\multicolumn{1}{|c|}{$\\bar{\\nu}_e$} & \\multicolumn{1}{c|}{%.1f} & \\multicolumn{1}{c|}{%.1f} & \\multicolumn{1}{c|}{%.1f} & \\multicolumn{1}{c|}{%.1f} & \\multicolumn{1}{c|}{%.1f} & \\multicolumn{1}{c|}{%.1f} & \\multicolumn{1}{c|}{%.2f} & \\multicolumn{1}{c|}{%.1f} & \\multicolumn{1}{c|}{%.1f} & \\multicolumn{1}{c|}{%.1f} & \\multicolumn{1}{c|}{%.1f} & \\multicolumn{1}{c|}{%.1f} \\\\ \\hline"%(N_s[3, 0], N_s[3, 1], N_s[3, 2], N_m[3, 0], N_m[3, 1], N_m[3, 2], N_h[3, 0], N_h[3, 1], N_h[3, 2], N[3, 0], N[3, 1], N[3, 2]))
    print("\\multicolumn{1}{|c|}{$\\bar{\\nu}_\\mu$} & \\multicolumn{1}{c|}{%.1f} & \\multicolumn{1}{c|}{%.1f} & \\multicolumn{1}{c|}{%.1f} & \\multicolumn{1}{c|}{%.1f} & \\multicolumn{1}{c|}{%.1f} & \\multicolumn{1}{c|}{%.1f} & \\multicolumn{1}{c|}{%.2f} & \\multicolumn{1}{c|}{%.1f} & \\multicolumn{1}{c|}{%.1f} & \\multicolumn{1}{c|}{%.1f} & \\multicolumn{1}{c|}{%.1f} & \\multicolumn{1}{c|}{%.1f} \\\\ \\hline"%(N_s[4, 0], N_s[4, 1], N_s[4, 2], N_m[4, 0], N_m[4, 1], N_m[4, 2], N_h[4, 0], N_h[4, 1], N_h[4, 2], N[4, 0], N[4, 1], N[4, 2]))
    print("\\multicolumn{1}{|c|}{$\\bar{\\nu}_\\tau$} & \\multicolumn{1}{c|}{%.1f} & \\multicolumn{1}{c|}{%.1f} & \\multicolumn{1}{c|}{%.1f} & \\multicolumn{1}{c|}{%.1f} & \\multicolumn{1}{c|}{%.1f} & \\multicolumn{1}{c|}{%.1f} & \\multicolumn{1}{c|}{%.2f} & \\multicolumn{1}{c|}{%.1f} & \\multicolumn{1}{c|}{%.1f} & \\multicolumn{1}{c|}{%.1f} & \\multicolumn{1}{c|}{%.1f} & \\multicolumn{1}{c|}{%.1f} \\\\ \\hline"%(N_s[5, 0], N_s[5, 1], N_s[5, 2], N_m[5, 0], N_m[5, 1], N_m[5, 2], N_h[5, 0], N_h[5, 1], N_h[5, 2], N[5, 0], N[5, 1], N[5, 2]))
    print("\\multicolumn{1}{|l|}{Total} & \\multicolumn{1}{l|}{%.1f} & \\multicolumn{1}{l|}{%.1f} & \\multicolumn{1}{l|}{%.1f} & \\multicolumn{1}{l|}{%.1f} & \\multicolumn{1}{l|}{%.1f} & \\multicolumn{1}{l|}{%.1f} & \\multicolumn{1}{l|}{%.1f} & \\multicolumn{1}{l|}{%.1f} & \\multicolumn{1}{l|}{%.2f} & \\multicolumn{1}{l|}{%.1f} & \\multicolumn{1}{l|}{%.1f} & \\multicolumn{1}{l|}{%.1f} \\\\ \\hline"%(N_s[6, 0], N_s[6, 1], N_s[6, 2], N_m[6, 0], N_m[6, 1], N_m[6, 2], N_h[6, 0], N_h[6, 1], N_h[6, 2], N[6, 0], N[6, 1], N[6, 2]))
    print("\\end{tabular}\n\\end{table}")
    
def events_table(data):
    data['final_weight'] = data.final_oscillated_w/40 #File is made for 400kt.yr, dividing by 40 to quote numbers for 10kt.yr
    df = pl.from_dataframe(data)

    df = df.with_columns(
        Ebin = pl.when(pl.col('Ev') < 1).then(
            pl.lit("Sub-GeV")
        ).otherwise(
            pl.when(
                pl.col('Ev') < 10
            ).then(
               pl.lit("Multi-GeV")
            ).otherwise(
                pl.lit("High-GeV")
            )
        )
    )
    
    return df.group_by(['nuPDG', 'isCC', 'Ebin']).agg(
        pl.sum('final_weight')
    )

def replace_empty_list(x, replacement):
    if len(x) == 0:
        return [replacement]
    return x




with uproot.open('/pnfs/dune/persistent/users/pgranger/atmospherics-data/atmospherics_prod_1M_events_cafs_hadd_with_weights.root') as f:
    weights = f['weights'].arrays(library='pd')
    weights['nuPDG'] = ak.flatten(f['cafTree/rec/mc/mc.nu.pdg'].array())
    weights['Ev'] = ak.flatten(f['cafTree/rec/mc/mc.nu.E'].array())
    weights['isCC'] = ak.flatten(f['cafTree/rec/mc/mc.nu.iscc'].array())
    weights['NuMomY'] = ak.flatten(f['cafTree/rec/mc/mc.nu.momentum.y'].array())
    weights['mode'] = ak.flatten(f['cafTree/rec/mc/mc.nu.mode'].array())
    # weights['recoE'] = ak.flatten(f['cafTree/rec/common/common.ixn.pandora/common.ixn.pandora.Enu.calo'].array())[1:]
    recoE = f['cafTree/rec/common/common.ixn.pandora/common.ixn.pandora.Enu.lep_calo'].array()
    # recoE = f['cafTree/rec/common/common.ixn.pandora/common.ixn.pandora.Enu.calo'].array()
    recoE = ak.Array([replace_empty_list(x, -999) for x in recoE])
    weights['recoE'] = ak.flatten(recoE)

    direc = f['cafTree/rec/common/common.ixn.pandora/common.ixn.pandora.dir.lngtrk.y'].array()
    # direc = f['cafTree/rec/common/common.ixn.pandora/common.ixn.pandora.dir.heshw.y'].array()
    direc = ak.Array([replace_empty_list(x, -999) for x in direc])
    weights['direc'] = ak.flatten(direc)
    # print(f['cafTree/rec/common/common.ixn.pandora'].keys())
df = events_table(weights)
PrintTable(df)

\begin{table}[h!]
\centering
\label{tab:label}
\begin{tabular}{ccccccccccccc}
\multicolumn{1}{l}{} & \multicolumn{12}{c}{\textbf{Totals per 10 kt $\cdot$ yr}} \\ \cline{2-13}
\multicolumn{1}{c|}{} & \multicolumn{12}{c|}{Solar Maximum, NuFIT 5.2, AR2320i} \\ \cline{2-13}
\multicolumn{1}{c|}{} & \multicolumn{3}{c|}{\begin{tabular}[c]{@{}c@{}}Sub-GeV\\ $[0.1-1.0]\,$GeV\end{tabular}} & \multicolumn{3}{c|}{\begin{tabular}[c]{@{}c@{}}Multi-GeV\\ $[1.0-10.0]\,$GeV\end{tabular}} & \multicolumn{3}{c|}{\begin{tabular}[c]{@{}c@{}}High-GeV\\ $[10.0-100.0]\,$GeV\end{tabular}} & \multicolumn{3}{c|}{\begin{tabular}[c]{@{}c@{}}Total\\ $[0.1-100.0]\,$GeV\end{tabular}} \\ \cline{2-13} 
\multicolumn{1}{c|}{} & \multicolumn{1}{c|}{CC} & \multicolumn{1}{c|}{NC} & \multicolumn{1}{c|}{Total} & \multicolumn{1}{c|}{CC} & \multicolumn{1}{c|}{NC} & \multicolumn{1}{c|}{Total} & \multicolumn{1}{c|}{CC} & \multicolumn{1}{c|}{NC} & \multicolumn{1}{c|}{Total} & \multicolumn{1}{c|}{CC} & \multicolumn{1}{c|}{NC} & \mul